# Generate Notebooks and TOC yaml file

1. Generates FRED template based on category_ids
2. Generates Yahoo Currency and Metals template based on tickers

## TOC yml file start

In [3]:
text = "format: jb-book\nroot: businesscycles/intro\nparts:\n"

# FRED

In [4]:
from dotenv import load_dotenv
import requests
import pandas as pd
import datetime
from tqdm import tqdm
import os
load_dotenv()
api_key = os.getenv("APIKEY")
print(api_key)

39f4f960ae8ea7fc0c370021ddf49544


## Favorites

In [5]:
text = text + "- caption: Favorites\n  chapters:\n"
for series_identifier in tqdm( ['T10YFF','T5YFF','T1YFF','T6MFF','T3MFF','DGS10','T10Y3M','T10Y2Y','T10YIE','T5YIE','MORTGAGE30US','DTWEXBGS','BOGMBASE','CPALTT01USM659N','GFDEGDQ188S','SP500','VIXCLS'] ):
    text = text + "  - file: businesscycles/fredapi/" + series_identifier + "\n"

100%|██████████| 17/17 [00:00<00:00, 99308.03it/s]


## Functions

In [6]:
def get_fred_category(category_id,api_key):
    import requests
    url = "https://api.stlouisfed.org/fred/category?category_id=" + str(category_id) + "&api_key=" + api_key + "&file_type=json"
    result = requests.get(url).json()['categories'][0]
    return(result)

In [7]:
def get_fred_series_in_category(category_id, api_key):
    import requests
    url = "https://api.stlouisfed.org/fred/category/series?category_id=" + str(category_id) + "&api_key=" + api_key + "&file_type=json&order_by=popularity&sort_order=desc&limit=5"
    prior_date = datetime.datetime.now() - datetime.timedelta(days=3*365)
    df = pd.DataFrame(data = requests.get(url).json()['seriess'] )
    df = df.query("last_updated >= '" + str(prior_date) + "'")
    result = { 'category_id' : category_id, 'data' : df.to_dict(orient="records") }
    return(result)

In [8]:
def get_fred_data(series_id,api_key):
    import requests
    url = "https://api.stlouisfed.org/fred/series/observations?series_id=" + series_id + "&api_key=" + api_key + "&file_type=json"
    df = pd.DataFrame(data = requests.get(url).json()['observations'] )
    df = df.drop(['realtime_start','realtime_end'],axis=1)
    df['value'] = pd.to_numeric(df["value"], downcast="float")
    df = df.rename(columns={"date": "ds", "value": "y"})
    return(df)


In [9]:
def generate_notebook(symbol,name):
    import nbformat as nbf
    nb = nbf.v4.new_notebook()

    # Title
    text = """# ({ticker}): {name}""".format(ticker = symbol, name = name)
    # Code
    code = """\
    from functions import *
    import os
    from dotenv import load_dotenv
    load_dotenv()
    apikey = os.getenv("APIKEY")
    fred = Fred(api_key=apikey)
    collect_tune_and_predict('{ticker}',fred,2*365)""".format(ticker = symbol)

    # Cells
    nb['cells'] = [nbf.v4.new_markdown_cell(text),
                nbf.v4.new_code_cell(code)]

    fname = 'businesscycles/fredapi/{ticker}.ipynb'.format(ticker = symbol)

    with open(fname, 'w') as f:
        nbf.write(nb, f)
    return True

## Pipeline

In [10]:
category_ids = sorted( list( set( [116,33120,94,114,32348,33446,124,30,28,32242,123,32218,32255,93,101,32457,32440,32242,33119,32217,110,33446,9,32240,33913,33119,32457,32425,32255,281,32240,33963,33972,33978,33980,32351,33509,12,32305,32325,32250,33500,33001] ) ) )

### Generate 1) Jupyter notebooks, 2) append to toc YMLs

In [11]:
# for category_id in tqdm( category_ids ):
#     ## Category header
#     category_name = get_fred_category(category_id,api_key)['name']
#     ## Results
#     category_result = get_fred_series_in_category(category_id,api_key)
#     ## No discontinued
#     for series in reversed( range(len(category_result['data']))):
#         if "DISCONTINUED" in category_result['data'][series]['title']:
#             del category_result['data'][series]
#     ## Text
#     if len(category_result['data'])==0:
#         continue
#     else:
#         text = text + "- caption: " + category_name + " (" + str(category_id) + ")\n  chapters:\n"
#         for series in category_result['data']:
#             generate_notebook(symbol = series['id'], name = series['title'] ) # generate ipynb files
#             text = text + "  - file: businesscycles/fredapi/{ticker}\n".format(ticker=series['id'])

# DST

In [12]:
series_dst = ['BYGV88','EJ14']

In [13]:
text = text + "- caption: DST\n  chapters:\n"
for dst in series_dst:
  text = text + "  - file: businesscycles/dst/{ticker}.\n".format(ticker=dst)

# Yahoo

In [14]:
tickers_metal = ['ALI','GC','HG','PA','PL','SI','CL','HO','NG','ZC','ZO','ZR','ZL','GF','HE','LE','CC','KC','CT','OJ','SB']
tickers_currencies = ['CHFDKK','GBPDKK','NOKDKK','SEKDKK','USDDKK','AUDDKK','NZDDKK','JPYDKK','CADDKK','HUFDKK','CNYDKK','HKDDKK','SGDDKK','INRDKK','MXNDKK','IDRDKK','RUBDKK','ZARDKK','MYRDKK','THBDKK']

## Metals

In [15]:
text = text + "- caption: Yahoo Metals\n  chapters:\n"
for metal in tickers_metal:
  text = text + "  - file: businesscycles/yahoo_metals/{ticker}\n".format(ticker=metal)

## Currencies

In [16]:
text = text + "- caption: Yahoo Currency\n  chapters:\n"
for metal in tickers_currencies:
    text = text + "  - file: businesscycles/yahoo_currencies/{ticker}\n".format(ticker=metal)

## Write TOC file

In [17]:
# export to YML
with open("_toc.yml", 'r+') as f:
    f.seek(0)
    f.write(text)
    f.truncate()